# Data Analysis with Python 

## Dr Andrew McCluskey 

#### Email: andrew.mccluskey@diamond.ac.uk

It is important to note that those represent only an example of how the exercise may be approached. Programming problems such as these have **many** correct methods.

We **strongly** advise that you try and work though the exercise without looking at this worked example. This is the best way to improve your programming skills; as programming is inherently problem-solving based.

#### Importing necessary libraries 

Here we import the libraries that are necessary to complete the exercise.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

#### Reading in and plotting experimental data

`np.loadtxt` is used to read in the model datasets and these are then plotted.

In [ ]:
toluene = np.loadtxt('./toluene.csv', unpack=True, delimiter=',')
plt.errorbar(toluene[0], toluene[1])
plt.xlabel('Wavennumber/cm$^{-1}$')
plt.ylabel('Transmittance')
plt.show()

In [ ]:
benzyl_alcohol = np.loadtxt('./benzyl_alcohol.csv', unpack=True, delimiter=',')
plt.errorbar(benzyl_alcohol[0], benzyl_alcohol[1])
plt.xlabel('Wavennumber/cm$^{-1}$')
plt.ylabel('Transmittance')
plt.show()

The same function is then used to read in the experimentally measured data, however now the data includes an experimental uncertainty therefore the `plt.errorbar` function is used for visualisation.

In [ ]:
mixture = np.loadtxt('./mixture.csv', unpack=True, delimiter=',')
plt.errorbar(mixture[0], mixture[1], mixture[2])
plt.xlabel('Wavennumber/cm$^{-1}$')
plt.ylabel('Transmittance')
plt.show()

#### Interpolation

First we assess the minimum and maximum on the $x$-axis for for each of the datasets to ensure that the the mixture data is within the model datsets.

In [ ]:
print("Model 1: min = {:.1f} cm-1, max = {:.1f} cm-1".format(np.min(benzyl_alcohol[0]), np.max(benzyl_alcohol[0])))
print("Model 2: min = {:.1f} cm-1, max = {:.1f} cm-1".format(np.min(toluene[0]), np.max(toluene[0])))
print("Mixture: min = {:.1f} cm-1, max = {:.1f} cm-1".format(np.min(mixture[0]), np.max(mixture[0])))

The interpolation axis is then defined. 

In [ ]:
optimisation_x = mixture[0]

The two model datatsets are then interpolated using the above defined axis. A new pair of arrays are then created for the two models, to be used in the optimisation of the model.

In [ ]:
interp_benzyl_alcohol = np.interp(optimisation_x, benzyl_alcohol[0], benzyl_alcohol[1])
interp_toulene = np.interp(optimisation_x, toluene[0], toluene[1])

opt_benzyl_alcohol = np.array([optimisation_x, interp_benzyl_alcohol])
opt_toluene = np.array([optimisation_x, interp_toulene])

These new arrays are then plotted to check that they appear correct. 

In [ ]:
plt.errorbar(opt_toluene[0], opt_toluene[1])
plt.errorbar(opt_benzyl_alcohol[0], opt_benzyl_alcohol[1])
plt.xlabel('Wavennumber/cm$^{-1}$')
plt.ylabel('Transmittance')
plt.show()

#### Fitting real data

We can now synthesize the model data for the mixture from the individual components.

In [ ]:
def model(c, T1, T2):
    """
    The model function.
    
    Args:
        c (float): The relative concentration of the 
            two components of the mixture.
        T1 (array_like, float): The transmittance of 
            component 1.
        T2 (array_like, float): The transmittance of 
            component 2.
    
    Returns:
        array_like, float: The transmittance of the 
            mixture.
    """
    return c * T1 + (1 - c) * T2

In [ ]:
plt.errorbar(mixture[0], mixture[1], mixture[2])
plt.errorbar(optimisation_x, model(0.5, opt_toluene[1], opt_benzyl_alcohol[1]))
plt.xlabel('Wavennumber/cm$^{-1}$')
plt.ylabel('Transmittance')
plt.show()

The goodness-of-fit metric can also be defined. 

In [ ]:
def goodness_of_fit(c, T1, T2, exp_T, dexp_T):
    """
    The goodness-of-fit metric between the model 
    and experimental data. 
    
    Args:
        c (float): The relative concentration of the 
            two components of the mixture.
        T1 (array_like, float): The transmittance of 
            component 1.
        T2 (array_like, float): The transmittance of 
            component 2.
        exp_T (array_like, float): The transmittance 
            of the mixture.
        dexp_T (array_like, float): The uncertainty in 
            transmittance of the mixture.
    
    Returns:
        float: The value of the chi-squared 
            goodness-of-fit                      
    """
    mod = model(c, T1, T2)
    chi_squared = np.sum(np.square((
        mod - exp_T) / dexp_T))
    return chi_squared

We can then minimise the function above to obtain the best agreement between the model and the experimental data. 

In [ ]:
from scipy.optimize import minimize
result = minimize(chi, 0.5, args=(opt_toluene[1], opt_benzyl_alcohol[1], mixture[1], mixture[2]))

We can plot this to observe the similarity. 

In [ ]:
plt.errorbar(mixture[0], mixture[1], mixture[2])
plt.errorbar(optimisation_x, model(result.x, opt_toluene[1], opt_benzyl_alcohol[1]))
plt.xlabel('Wavennumber/cm$^{-1}$')
plt.ylabel('Transmittance')
plt.show()

#### Markov chain Monte Carlo

Now we defined the MCMC algorithm. 

In [ ]:
def mcmc(c, model, step_size, iterations, model1, model2, exp):
    """
    An implementation of the MCMC algorithm.
    
    Args:
        theta (float): The initial guess of the 
            best fit.
        model (function): The funciton used to find 
            model transmittance.
        step_size (float): The size of the pertubation.
        iterations (int): The number of iterations to be 
            performed.
        model1 (array_like, float): The y-axis data for model 1.
        model2 (array_like, float): The y-axis data for model 2.
        exp (array_like, float): The experimental data (x-, y-, 
            and dy-axes).
    
    Returns:
        array_like, float: An array of all of the allowing values 
            for c.
    """
    accepted = []
    chi_squared =  goodness_of_fit(c, model1, model2, exp[1], exp[2])
    for i in range(0, iterations):
        perturbation = step_size * np.random.randn() * c
        new_c = c + perturbation
        new_chi_squared =  goodness_of_fit(new_c, model1, model2, 
                                           exp[1], exp[2])
        p = np.exp((-new_chi_squared + chi_squared) / 2)
        n = np.random.random()
        if n < p:
            c = new_c
            chi_squared = new_chi_squared
            accepted.append(c)
    return np.array(accepted)

It is then possible to run the above algorithm, plot the histogram of the allowed values and print the relative concentration with an uncertainty. 

In [ ]:
allowed_c = mcmc(result.x[0], model, 0.001, 10000, opt_toluene[1], opt_benzyl_alcohol[1], mixture)

In [ ]:
plt.hist(allowed_c, density=True)
plt.xlabel('Relative concentration')
plt.ylabel('Relative probability')
plt.show()

In [ ]:
print("The relative concentation is {:.3f} +/- {:.3f}.".format(
    np.mean(allowed_c), np.std(allowed_c)))